<a href="https://colab.research.google.com/github/Arif2455/lung-tumour-detection/blob/main/LungTumourDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Cell 1: configure these variables ===
UPLOAD_DIR = "/content"            # where your loose images currently are (change if different)
DATA_DIR   = "/content/lung_project"  # target dataset folder we'll create
CLASS_A_NAME = "normal"            # change if you want different names
CLASS_B_NAME = "tumour"
CLASS_A_COUNT = 154                # number of images you want in CLASS_A
CLASS_B_COUNT = 93                 # number of images you want in CLASS_B
SPLIT_MODE = "by_count"            # options: "by_count", "by_pattern", "by_csv"

# === Helper & quick preview ===
import pathlib, os
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff'}
p = pathlib.Path(UPLOAD_DIR)
all_imgs = [str(x) for x in sorted(p.glob("*")) if x.suffix.lower() in IMG_EXTS]

print(f"Found {len(all_imgs)} image files in {UPLOAD_DIR}")
print("First 40 filenames:")
for i,fn in enumerate(all_imgs[:40], 1):
    print(f"{i:3d}. {os.path.basename(fn)}")


Found 246 image files in /content
First 40 filenames:
  1. JPCLN001.png
  2. JPCLN002.png
  3. JPCLN003.png
  4. JPCLN004.png
  5. JPCLN005.png
  6. JPCLN006.png
  7. JPCLN007.png
  8. JPCLN008.png
  9. JPCLN009.png
 10. JPCLN010.png
 11. JPCLN011.png
 12. JPCLN012.png
 13. JPCLN013.png
 14. JPCLN014.png
 15. JPCLN015.png
 16. JPCLN016.png
 17. JPCLN017.png
 18. JPCLN018.png
 19. JPCLN019.png
 20. JPCLN020.png
 21. JPCLN021.png
 22. JPCLN022.png
 23. JPCLN023.png
 24. JPCLN024.png
 25. JPCLN025.png
 26. JPCLN026.png
 27. JPCLN027.png
 28. JPCLN028.png
 29. JPCLN029.png
 30. JPCLN030.png
 31. JPCLN031.png
 32. JPCLN032.png
 33. JPCLN033.png
 34. JPCLN034.png
 35. JPCLN035.png
 36. JPCLN036.png
 37. JPCLN037.png
 38. JPCLN038.png
 39. JPCLN039.png
 40. JPCLN040.png


In [ ]:
# === Cell 2: create folder structure ===
import os, shutil, math
os.makedirs(DATA_DIR, exist_ok=True)
class_a_dir = os.path.join(DATA_DIR, CLASS_A_NAME)
class_b_dir = os.path.join(DATA_DIR, CLASS_B_NAME)
os.makedirs(class_a_dir, exist_ok=True)
os.makedirs(class_b_dir, exist_ok=True)
print("Created folders:")
print(" -", class_a_dir)
print(" -", class_b_dir)


Created folders:
 - /content/lung_project/normal
 - /content/lung_project/tumour


In [ ]:
# === Cell 3A: Auto split by count ===
if SPLIT_MODE != "by_count":
    print("Skipping by_count (SPLIT_MODE != 'by_count').")
else:
    imgs = all_imgs.copy()
    total_needed = CLASS_A_COUNT + CLASS_B_COUNT
    if len(imgs) < total_needed:
        raise SystemExit(f"Not enough images found ({len(imgs)}) for requested counts ({total_needed}).")
    # Move first N to class A, next M to class B
    a_list = imgs[:CLASS_A_COUNT]
    b_list = imgs[CLASS_A_COUNT:CLASS_A_COUNT+CLASS_B_COUNT]

    import shutil, os
    for src in a_list:
        dst = os.path.join(class_a_dir, os.path.basename(src))
        shutil.move(src, dst)
    for src in b_list:
        dst = os.path.join(class_b_dir, os.path.basename(src))
        shutil.move(src, dst)

    print(f"Moved {len(a_list)} → {class_a_dir}")
    print(f"Moved {len(b_list)} → {class_b_dir}")


SystemExit: Not enough images found (246) for requested counts (247).

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import shutil, os

CLASS_A_NAME = "normal"
CLASS_B_NAME = "tumour"
CLASS_A_COUNT = 153   # adjust so A+B = total
CLASS_B_COUNT = 93

# Source = all loose images in /content
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff'}
import pathlib
UPLOAD_DIR = "/content"
all_imgs = [str(x) for x in sorted(pathlib.Path(UPLOAD_DIR).glob("*")) if x.suffix.lower() in IMG_EXTS]

print(f"Total images found: {len(all_imgs)}")

# Make folders
DATA_DIR = "/content/lung_project"
os.makedirs(os.path.join(DATA_DIR, CLASS_A_NAME), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, CLASS_B_NAME), exist_ok=True)

# Split
a_list = all_imgs[:CLASS_A_COUNT]
b_list = all_imgs[CLASS_A_COUNT:CLASS_A_COUNT+CLASS_B_COUNT]

for src in a_list:
    dst = os.path.join(DATA_DIR, CLASS_A_NAME, os.path.basename(src))
    shutil.move(src, dst)

for src in b_list:
    dst = os.path.join(DATA_DIR, CLASS_B_NAME, os.path.basename(src))
    shutil.move(src, dst)

print(f"✅ Moved {len(a_list)} images → {CLASS_A_NAME}")
print(f"✅ Moved {len(b_list)} images → {CLASS_B_NAME}")


Total images found: 246
✅ Moved 153 images → normal
✅ Moved 93 images → tumour


In [ ]:
import pathlib

for folder in pathlib.Path(DATA_DIR).iterdir():
    if folder.is_dir():
        count = len(list(folder.glob("*")))
        print(f"{folder.name}: {count} images")


tumour: 93 images
normal: 153 images


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2, # 80% train, 20% val
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Cache + prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Build simple CNN
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224,224,3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Found 246 files belonging to 2 classes.
Using 197 files for training.
Found 246 files belonging to 2 classes.
Using 49 files for validation.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4732 - loss: 1.1181 - val_accuracy: 0.5714 - val_loss: 0.6856
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6065 - loss: 0.6685 - val_accuracy: 0.5714 - val_loss: 0.6851
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6220 - loss: 0.6758 - val_accuracy: 0.5714 - val_loss: 0.7009
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5892 - loss: 0.6845 - val_accuracy: 0.5714 - val_loss: 0.7623
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6337 - loss: 0.6682 - val_accuracy: 0.5714 - val_loss: 0.6925
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6464 - loss: 0.6552 - val_accuracy: 0.5714 - val_loss: 0.6835
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5897 - loss: 0.6766 - val_accuracy: 0.5714 - val_loss: 0.7099
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6373 - loss: 0.6409 - val_accuracy: 0.5714 - val_loss:

In [ ]:
MODEL_PATH = "/content/lung_tumour_model.h5"
model.save(MODEL_PATH)
print("✅ Model saved at:", MODEL_PATH)


✅ Model saved at: /content/lung_tumour_model.h5


In [ ]:
import os

print("Tumour samples:")
print(os.listdir("/content/lung_project/tumour")[:5])

print("\nNormal samples:")
print(os.listdir("/content/lung_project/normal")[:5])



Tumour samples:
['JPCNN066.png', 'JPCNN054.png', 'JPCNN020.png', 'JPCNN037.png', 'JPCNN082.png']

Normal samples:
['JPCLN096.png', 'JPCLN079.png', 'JPCLN021.png', 'JPCLN146.png', 'JPCLN019.png']


In [ ]:
import os

print("Tumour samples:", os.listdir("/content/lung_project/tumour")[:10])
print("\nNormal samples:", os.listdir("/content/lung_project/normal")[:10])



Tumour samples: ['JPCNN066.png', 'JPCNN054.png', 'JPCNN020.png', 'JPCNN037.png', 'JPCNN082.png', 'JPCNN089.png', 'JPCNN091.png', 'JPCNN033.png', 'JPCNN043.png', 'JPCNN006.png']

Normal samples: ['JPCLN096.png', 'JPCLN079.png', 'JPCLN021.png', 'JPCLN146.png', 'JPCLN019.png', 'JPCLN069.png', 'JPCLN020.png', 'JPCLN077.png', 'JPCLN042.png', 'JPCLN091.png']


In [ ]:
import os

print("Tumour samples:")
print(os.listdir("/content/lung_project/tumour")[:10])

print("\nNormal samples:")
print(os.listdir("/content/lung_project/normal")[:10])



Tumour samples:
['JPCNN066.png', 'JPCNN054.png', 'JPCNN020.png', 'JPCNN037.png', 'JPCNN082.png', 'JPCNN089.png', 'JPCNN091.png', 'JPCNN033.png', 'JPCNN043.png', 'JPCNN006.png']

Normal samples:
['JPCLN096.png', 'JPCLN079.png', 'JPCLN021.png', 'JPCLN146.png', 'JPCLN019.png', 'JPCLN069.png', 'JPCLN020.png', 'JPCLN077.png', 'JPCLN042.png', 'JPCLN091.png']


In [ ]:
# Test tumour
test_img = "/content/lung_project/tumour/JPCNN066.png"
print("Tumour test:", predict_image(test_img))

# Test normal
test_img = "/content/lung_project/normal/JPCLN096.png"
print("Normal test:", predict_image(test_img))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Tumour test: ('Normal', np.float32(0.20936929))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Normal test: ('Normal', np.float32(0.21083398))


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Pretrained base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze feature extractor

# Build model
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

MODEL_PATH = "/content/lung_tumour_mobilenet.h5"
model.save(MODEL_PATH)
print("✅ Transfer learning model saved at:", MODEL_PATH)


Found 246 files belonging to 2 classes.
Using 197 files for training.
Found 246 files belonging to 2 classes.
Using 49 files for validation.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.6395 - loss: 0.7207 - val_accuracy: 0.5714 - val_loss: 0.6454
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 768ms/step - accuracy: 0.6504 - loss: 0.6677 - val_accuracy: 0.5918 - val_loss: 0.7027
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 771ms/step - accuracy: 0.6329 - loss: 0.7084 - val_accuracy: 0.6122 - val_loss: 0.6783
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 782ms/step - accuracy: 0.6579 - loss: 0.6217 - val_accuracy: 0.6327 - val_loss: 0.6673
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 679ms/step - accuracy: 0.6320 - loss: 0.6476 - val_accuracy: 0.6531 - val_loss: 0.6386
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 908ms/step - accuracy: 0.6244 - loss: 0.6581 - val_accuracy: 0.6735 - val_loss: 0.6316
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 19

✅ Transfer learning model saved at: /content/lung_tumour_mobilenet.h5


In [ ]:
print(predict_image("/content/lung_project/tumour/JPCNN066.png"))
print(predict_image("/content/lung_project/normal/JPCLN096.png"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
('Tumour', np.float32(0.65461606))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
('Tumour', np.float32(0.65422356))


In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load model
MODEL_PATH = "lung_tumour_mobilenet.h5"
model = tf.keras.models.load_model(MODEL_PATH)
IMG_SIZE = (224,224)

st.title("🫁 Lung Tumour Detection App")
st.write("Upload a lung scan image to classify as **Tumour** or **Normal**.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg","png","jpeg"])

def predict_image(img):
    img = load_img(img, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0][0]
    label = "Tumour" if pred > 0.5 else "Normal"
    return label, pred

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    label, score = predict_image(uploaded_file)
    st.write(f"### Prediction: {label}")
    st.write(f"Confidence Score: {float(score):.2f}")


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load trained model
MODEL_PATH = "lung_tumour_mobilenet.h5"
model = tf.keras.models.load_model(MODEL_PATH)
IMG_SIZE = (224,224)

st.title("🫁 Lung Tumour Detection")
st.write("Upload a lung scan image and classify it as **Tumour** or **Normal**.")

# File uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg","png","jpeg"])

def predict_image(img):
    img = load_img(img, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0][0]
    label = "Tumour" if pred > 0.5 else "Normal"
    return label, pred

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    label, score = predict_image(uploaded_file)
    st.write(f"### Prediction: {label}")
    st.write(f"Confidence: {float(score):.2f}")


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.5 MB/s eta 0:00:00


In [ ]:
import streamlit as st


In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load trained model
MODEL_PATH = "lung_tumour_mobilenet.h5"
model = tf.keras.models.load_model(MODEL_PATH)
IMG_SIZE = (224,224)

st.title("🫁 Lung Tumour Detection")
st.write("Upload a lung scan image and classify it as **Tumour** or **Normal**.")

# File uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg","png","jpeg"])

def predict_image(img):
    img = load_img(img, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0][0]
    label = "Tumour" if pred > 0.5 else "Normal"
    return label, pred

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    label, score = predict_image(uploaded_file)
    st.write(f"### Prediction: {label}")
    st.write(f"Confidence: {float(score):.2f}")


2025-09-03 16:24:01.604 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.012 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-03 16:24:02.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.017 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.018 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-03 16:24:02.019 Thread 'MainThread': mi

In [ ]:
from google.colab import files
files.download("lung_tumour_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install streamlit tensorflow pillow


In [ ]:
!streamlit run app.py



Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [ ]:
!pip install streamlit tensorflow pillow numpy


In [ ]:
streamlit run app.py


SyntaxError: invalid syntax (ipython-input-507122745.py, line 1)

In [ ]:
!streamlit run app.py


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# --- Data Augmentation ---
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Load base model
base_model = keras.applications.MobileNetV2(input_shape=(224,224,3),
                                            include_top=False,
                                            weights="imagenet")
base_model.trainable = False   # Step 1: freeze

# Build model
inputs = keras.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

# Compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Train frozen base
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)

# --- Fine-tuning ---
base_model.trainable = True
for layer in base_model.layers[:-50]:  # keep first layers frozen
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

history_finetune = model.fit(train_ds,
                             validation_data=val_ds,
                             epochs=10)

# --- Evaluate ---
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true, y_pred = [], []
for imgs, labels in val_ds:
    preds = model.predict(imgs)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype("int32").flatten())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["Normal","Tumour"]))


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.5177 - loss: 0.9286 - val_accuracy: 0.5306 - val_loss: 0.6883
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 911ms/step - accuracy: 0.5857 - loss: 0.7417 - val_accuracy: 0.7347 - val_loss: 0.6403
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 932ms/step - accuracy: 0.5354 - loss: 0.8008 - val_accuracy: 0.5714 - val_loss: 0.6552
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 890ms/step - accuracy: 0.5431 - loss: 0.7644 - val_accuracy: 0.5918 - val_loss: 0.6535
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 893ms/step - accuracy: 0.5999 - loss: 0.7216 - val_accuracy: 0.5714 - val_loss: 0.6573
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.5920 - loss: 0.6987 - val_accuracy: 0.5714 - val_loss: 0.6523
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4330 - loss: 0.7511 - val_accuracy: 0.5918 - val_loss: 0.6512
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4883 - loss: 0.7584 - val_accuracy: 0.5918 - va

In [ ]:
model.save(r"C:\lung_app\lung_tumour_model_v2.keras")


In [ ]:
# Save model in keras format inside lung_app
model.save(r"C:\lung_app\lung_tumour_model_v2.keras")


In [ ]:
model = tf.keras.models.load_model("lung_tumour_model_v2.keras")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 55 variables whereas the saved optimizer has 108 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.save("lung_tumour_model_v2.keras")
from google.colab import files
files.download("lung_tumour_model_v2.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model("lung_tumour_model_v2.keras")


ValueError: File not found: filepath=lung_tumour_model_v2.keras. Please ensure the file is an accessible `.keras` zip file.

In [2]:
import os
print(os.listdir())


['.config', 'sample_data']


In [3]:
from google.colab import files
uploaded = files.upload()  # Select your lung_tumour_model_v2.keras file


Saving lung_tumour_model_v2.keras to lung_tumour_model_v2.keras


In [4]:
import os
print(os.listdir())


['.config', 'lung_tumour_model_v2.keras', 'sample_data']


In [6]:
from tensorflow import keras
model = keras.models.load_model("lung_tumour_model_v2.keras")  # or .h5 if that’s what you uploaded


In [7]:
# 1️⃣ Check the files in Colab
import os
print(os.listdir())  # You should see your .keras or .h5 file here

# 2️⃣ Load the model
from tensorflow import keras

model = keras.models.load_model("lung_tumour_model_v2.keras")  # or .h5 if that’s the file you uploaded

# 3️⃣ Check the model summary
model.summary()

# 4️⃣ (Optional) Evaluate accuracy if you have a test dataset
# Example:
# test_loss, test_acc = model.evaluate(x_test, y_test)
# print("Test Accuracy:", test_acc)


['.config', 'lung_tumour_model_v2.keras', 'sample_data']


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,115,652 (15.70 MB)

 Trainable params: 1,856,385 (7.08 MB)

 Non-trainable params: 402,880 (1.54 MB)

 Optimizer params: 1,856,387 (7.08 MB)

In [8]:
# 1️⃣ Import libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# 2️⃣ Load your model
model = keras.models.load_model("lung_tumour_model_v2.keras")  # make sure the file exists

# 3️⃣ Prepare your test dataset
# Example: if your test images are in 'test_images/' folder with subfolders 'Normal' and 'Tumour'
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir = 'test_images'  # replace with your folder path
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # or 'categorical' if more than 2 classes
    shuffle=False
)

# 4️⃣ Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")


FileNotFoundError: [Errno 2] No such file or directory: 'test_images'

In [1]:
from google.colab import files
uploaded = files.upload()  # select your zip file



Saving converted_png-20250903T150611Z-1-001.zip to converted_png-20250903T150611Z-1-001.zip


In [5]:
import zipfile

# Extract the uploaded zip
with zipfile.ZipFile("converted_png-20250903T150611Z-1-001.zip", 'r') as zip_ref:
    zip_ref.extractall("test_images")  # Extract into a folder named 'test_images'

print("Extraction completed!")


Extraction completed!


In [7]:
from google.colab import files
uploaded = files.upload()  # Select lung_tumour_model_v2.keras from your PC

Saving lung_tumour_model_v2.keras to lung_tumour_model_v2.keras


In [8]:
!ls


converted_png-20250903T150611Z-1-001.zip  sample_data
lung_tumour_model_v2.keras		  test_images


In [9]:
import tensorflow as tf

model = tf.keras.models.load_model("lung_tumour_model_v2.keras")


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'test_images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Found 246 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 1.0000 - loss: 0.4171
Test Accuracy: 100.00%


In [11]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load trained model
MODEL_PATH = "lung_tumour_mobilenet.h5"
model = tf.keras.models.load_model(MODEL_PATH)
IMG_SIZE = (224,224)

st.title("🫁 Lung Tumour Detection")
st.write("Upload a lung scan image and classify it as **Tumour** or **Normal**.")

# File uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg","png","jpeg"])

def predict_image(img):
    img = load_img(img, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0][0]
    label = "Tumour" if pred > 0.5 else "Normal"
    return label, pred

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    label, score = predict_image(uploaded_file)
    st.write(f"### Prediction: {label}")
    st.write(f"Confidence: {float(score):.2f}")


Writing app.py


In [15]:
import os
os.listdir()


['.config',
 'test_images',
 'app.py',
 'converted_png-20250903T150611Z-1-001.zip',
 'lung_tumour_model_v2.keras',
 'sample_data']

In [16]:
import tensorflow as tf

# Load dataset (assuming all images are in subfolders by class inside test_images/)
val_ds = tf.keras.utils.image_dataset_from_directory(
    'test_images/',
    image_size=(224, 224),  # adjust if your model expects a different size
    batch_size=32
)

# Evaluate your model
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


Found 246 files belonging to 1 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.5768 - loss: 0.6657
Validation Accuracy: 56.10%


In [20]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'test_images/',        # your actual folder
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'test_images/',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


Found 246 files belonging to 1 classes.
Using 197 files for training.
Found 246 files belonging to 1 classes.
Using 49 files for validation.


In [22]:
import os

os.listdir('test_images/')


['converted_png']

In [23]:
import os
import shutil

# Create dataset directories if they don't exist
os.makedirs('dataset/tumor', exist_ok=True)
os.makedirs('dataset/normal', exist_ok=True)

# List all files in test_images
files = os.listdir('test_images')

for file in files:
    src = os.path.join('test_images', file)

    # Skip if it's not a file
    if not os.path.isfile(src):
        continue

    # Decide destination folder
    if 'tumor' in file.lower():
        dst = os.path.join('dataset/tumor', file)
    else:
        dst = os.path.join('dataset/normal', file)

    shutil.move(src, dst)  # move file


In [25]:
import os

print("Contents of dataset/:", os.listdir('dataset'))
print("Contents of dataset/tumor/:", os.listdir('dataset/tumor'))
print("Contents of dataset/normal/:", os.listdir('dataset/normal'))


Contents of dataset/: ['tumor', 'normal']
Contents of dataset/tumor/: []
Contents of dataset/normal/: []


In [26]:
os.listdir('test_images')


['converted_png']

In [27]:
import shutil

src_folder = 'test_images'
dst_folder = 'dataset'

for file in os.listdir(src_folder):
    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
        if 'tumor' in file.lower():
            os.makedirs(os.path.join(dst_folder, 'tumor'), exist_ok=True)
            shutil.move(os.path.join(src_folder, file), os.path.join(dst_folder, 'tumor', file))
        else:
            os.makedirs(os.path.join(dst_folder, 'normal'), exist_ok=True)
            shutil.move(os.path.join(src_folder, file), os.path.join(dst_folder, 'normal', file))


In [32]:
import zipfile
import os

# Put the filename as a string
zip_path = "converted_png-20250903T150611Z-1-001.zip"
extract_dir = "test_images"

# Extract the ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check the extracted files
print("Files extracted:", os.listdir(extract_dir))


Files extracted: ['converted_png']


In [33]:
import os
import shutil

# Paths
source_dir = "test_images"
dataset_dir = "dataset"
tumor_dir = os.path.join(dataset_dir, "tumor")
normal_dir = os.path.join(dataset_dir, "normal")

# Create dataset directories if they don't exist
os.makedirs(tumor_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Move images to corresponding folders
for file in os.listdir(source_dir):
    if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
        # Here we assume your tumor images contain 'tumor' in the filename
        if "tumor" in file.lower():
            shutil.move(os.path.join(source_dir, file), tumor_dir)
        else:
            shutil.move(os.path.join(source_dir, file), normal_dir)

# Verify
print("Tumor images:", os.listdir(tumor_dir))
print("Normal images:", os.listdir(normal_dir))


Tumor images: []
Normal images: []


In [35]:
import os

print("test_images contents:", os.listdir("test_images"))
print("dataset contents:", os.listdir("dataset") if os.path.exists("dataset") else "No dataset folder")


test_images contents: ['converted_png']
dataset contents: ['tumor', 'normal']


In [36]:
import os
import shutil

# Create dataset folders
os.makedirs("dataset/tumor", exist_ok=True)
os.makedirs("dataset/normal", exist_ok=True)

# Move images into the right folders
for file in os.listdir("test_images"):
    if file.endswith((".png", ".jpg", ".jpeg")):
        if "tumor" in file.lower():
            shutil.move(os.path.join("test_images", file), "dataset/tumor")
        else:
            shutil.move(os.path.join("test_images", file), "dataset/normal")

# Verify
print("Tumor images:", os.listdir("dataset/tumor"))
print("Normal images:", os.listdir("dataset/normal"))


Tumor images: []
Normal images: []


In [38]:
import zipfile
import os
import shutil

# 1️⃣ Extract ZIP into 'test_images'
with zipfile.ZipFile("converted_png-20250903T150611Z-1-001.zip", 'r') as zip_ref:
    zip_ref.extractall("test_images")

# 2️⃣ Create dataset folders
os.makedirs("dataset/tumor", exist_ok=True)
os.makedirs("dataset/normal", exist_ok=True)

# 3️⃣ Walk through all extracted files and move them
for root, dirs, files in os.walk("test_images"):
    for file in files:
        if file.endswith((".png", ".jpg", ".jpeg")):
            src_path = os.path.join(root, file)
            if "tumor" in file.lower():
                shutil.move(src_path, "dataset/tumor")
            else:
                shutil.move(src_path, "dataset/normal")

# 4️⃣ Verify
print("Tumor images:", os.listdir("dataset/tumor")[:10])
print("Normal images:", os.listdir("dataset/normal")[:10])


Tumor images: []
Normal images: ['JPCLN043.png', 'JPCNN026.png', 'JPCLN114.png', 'JPCLN104.png', 'JPCNN043.png', 'JPCLN046.png', 'JPCNN057.png', 'JPCLN101.png', 'JPCLN063.png', 'JPCLN020.png']


In [39]:
import os

for root, dirs, files in os.walk("test_images"):
    print("Folder:", root)
    print("Files:", files[:10])


Folder: test_images
Files: []
Folder: test_images/converted_png
Files: []


In [40]:
import os
import shutil

# Paths
source_folder = "test_images"
tumor_folder = "dataset/tumor"
normal_folder = "dataset/normal"

# Create folders if they don't exist
os.makedirs(tumor_folder, exist_ok=True)
os.makedirs(normal_folder, exist_ok=True)

# Move files based on filename
for file in os.listdir(source_folder):
    if file.endswith((".png", ".jpg", ".jpeg")):
        if file.startswith("JPCNN"):
            shutil.move(os.path.join(source_folder, file), tumor_folder)
        elif file.startswith("JPCLN"):
            shutil.move(os.path.join(source_folder, file), normal_folder)

print("Files organized!")
print("Tumor images:", os.listdir(tumor_folder))
print("Normal images:", os.listdir(normal_folder))


Files organized!
Tumor images: []
Normal images: ['JPCLN043.png', 'JPCNN026.png', 'JPCLN114.png', 'JPCLN104.png', 'JPCNN043.png', 'JPCLN046.png', 'JPCNN057.png', 'JPCLN101.png', 'JPCLN063.png', 'JPCLN020.png', 'JPCLN049.png', 'JPCLN144.png', 'JPCLN106.png', 'JPCLN118.png', 'JPCNN036.png', 'JPCNN010.png', 'JPCNN055.png', 'JPCNN054.png', 'JPCNN062.png', 'JPCLN082.png', 'JPCLN108.png', 'JPCLN098.png', 'JPCNN033.png', 'JPCLN069.png', 'JPCLN056.png', 'JPCNN045.png', 'JPCNN005.png', 'JPCLN037.png', 'JPCLN003.png', 'JPCNN058.png', 'JPCNN061.png', 'JPCLN026.png', 'JPCLN094.png', 'JPCLN023.png', 'JPCNN038.png', 'JPCLN129.png', 'JPCLN029.png', 'JPCNN052.png', 'JPCLN092.png', 'JPCLN147.png', 'JPCLN119.png', 'JPCLN018.png', 'JPCLN032.png', 'JPCLN071.png', 'JPCLN138.png', 'JPCLN142.png', 'JPCNN068.png', 'JPCLN068.png', 'JPCNN090.png', 'JPCLN002.png', 'JPCLN102.png', 'JPCLN125.png', 'JPCLN134.png', 'JPCNN080.png', 'JPCNN020.png', 'JPCNN001.png', 'JPCLN135.png', 'JPCNN077.png', 'JPCLN015.png', 'JPCNN

In [41]:
import os
import shutil

# Paths
source_folder = "test_images"
tumor_folder = "dataset/tumor"
normal_folder = "dataset/normal"

# Create folders
os.makedirs(tumor_folder, exist_ok=True)
os.makedirs(normal_folder, exist_ok=True)

# Move files into the correct folders
for file in os.listdir(source_folder):
    if file.endswith((".png", ".jpg", ".jpeg")):
        # Convert filename to uppercase to avoid case issues
        upper_file = file.upper()
        if upper_file.startswith("JPCNN"):  # Tumor
            shutil.move(os.path.join(source_folder, file), tumor_folder)
        elif upper_file.startswith("JPCLN"):  # Normal
            shutil.move(os.path.join(source_folder, file), normal_folder)

# Check results
print("Tumor images:", os.listdir(tumor_folder))
print("Normal images:", os.listdir(normal_folder))


Tumor images: []
Normal images: ['JPCLN043.png', 'JPCNN026.png', 'JPCLN114.png', 'JPCLN104.png', 'JPCNN043.png', 'JPCLN046.png', 'JPCNN057.png', 'JPCLN101.png', 'JPCLN063.png', 'JPCLN020.png', 'JPCLN049.png', 'JPCLN144.png', 'JPCLN106.png', 'JPCLN118.png', 'JPCNN036.png', 'JPCNN010.png', 'JPCNN055.png', 'JPCNN054.png', 'JPCNN062.png', 'JPCLN082.png', 'JPCLN108.png', 'JPCLN098.png', 'JPCNN033.png', 'JPCLN069.png', 'JPCLN056.png', 'JPCNN045.png', 'JPCNN005.png', 'JPCLN037.png', 'JPCLN003.png', 'JPCNN058.png', 'JPCNN061.png', 'JPCLN026.png', 'JPCLN094.png', 'JPCLN023.png', 'JPCNN038.png', 'JPCLN129.png', 'JPCLN029.png', 'JPCNN052.png', 'JPCLN092.png', 'JPCLN147.png', 'JPCLN119.png', 'JPCLN018.png', 'JPCLN032.png', 'JPCLN071.png', 'JPCLN138.png', 'JPCLN142.png', 'JPCNN068.png', 'JPCLN068.png', 'JPCNN090.png', 'JPCLN002.png', 'JPCLN102.png', 'JPCLN125.png', 'JPCLN134.png', 'JPCNN080.png', 'JPCNN020.png', 'JPCNN001.png', 'JPCLN135.png', 'JPCNN077.png', 'JPCLN015.png', 'JPCNN076.png', 'JPCLN1

In [44]:
import pathlib

image_dir = pathlib.Path("test_images/converted_png")  # ← update this path
print("Files found:", list(image_dir.glob("*.png")))  # sanity check


Files found: []


In [45]:
import zipfile
import os

# Path to the zip file
zip_path = "converted_png-20250903T150611Z-1-001.zip"
extract_path = "test_images/converted_png"  # folder to extract images to

# Make sure the folder exists
os.makedirs(extract_path, exist_ok=True)

# Extract zip contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check extracted files
print("Files extracted:", os.listdir(extract_path))


Files extracted: ['converted_png']


In [46]:
import os

# Path to the folder containing images
dataset_path = "test_images/converted_png/converted_png"

# Check files
print("Number of images found:", len(os.listdir(dataset_path)))
print("Some sample files:", os.listdir(dataset_path)[:10])


Number of images found: 246
Some sample files: ['JPCLN043.png', 'JPCNN026.png', 'JPCLN114.png', 'JPCLN104.png', 'JPCNN043.png', 'JPCLN046.png', 'JPCNN057.png', 'JPCLN101.png', 'JPCLN063.png', 'JPCLN020.png']


In [48]:
import os
import shutil

src_folder = "test_images/converted_png/converted_png"
dataset_folder = "dataset"
tumor_folder = os.path.join(dataset_folder, "tumor")
normal_folder = os.path.join(dataset_folder, "normal")

os.makedirs(tumor_folder, exist_ok=True)
os.makedirs(normal_folder, exist_ok=True)

for file in os.listdir(src_folder):
    if file.endswith((".png", ".jpg", ".jpeg")):
        src_path = os.path.join(src_folder, file)
        if file.startswith("JPCNN"):
            dst_path = os.path.join(tumor_folder, file)
        elif file.startswith("JPCLN"):
            dst_path = os.path.join(normal_folder, file)
        else:
            continue

        # If file exists, overwrite
        if os.path.exists(dst_path):
            os.remove(dst_path)
        shutil.move(src_path, dst_path)

print("Tumor images:", len(os.listdir(tumor_folder)))
print("Normal images:", len(os.listdir(normal_folder)))


Tumor images: 93
Normal images: 246


In [49]:
import random
import shutil
import os

normal_folder = "dataset/normal"
tumor_folder = "dataset/tumor"
balanced_normal_folder = "dataset/normal_balanced"
os.makedirs(balanced_normal_folder, exist_ok=True)

normal_images = os.listdir(normal_folder)
selected_images = random.sample(normal_images, len(os.listdir(tumor_folder)))

for file in selected_images:
    shutil.copy(os.path.join(normal_folder, file), os.path.join(balanced_normal_folder, file))

print("Balanced normal images:", len(os.listdir(balanced_normal_folder)))


Balanced normal images: 93


In [50]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


Found 432 files belonging to 3 classes.
Using 346 files for training.
Found 432 files belonging to 3 classes.
Using 86 files for validation.


In [51]:
class_names = train_ds.class_names
print("Classes:", class_names)


Classes: ['normal', 'normal_balanced', 'tumor']


In [52]:
import os
import shutil

# Paths
dataset_dir = "dataset"
normal_dir = os.path.join(dataset_dir, "normal")
normal_balanced_dir = os.path.join(dataset_dir, "normal_balanced")

# Move all images from normal_balanced to normal
for file in os.listdir(normal_balanced_dir):
    src_path = os.path.join(normal_balanced_dir, file)
    dst_path = os.path.join(normal_dir, file)
    if not os.path.exists(dst_path):  # Avoid overwriting
        shutil.move(src_path, dst_path)

# Remove the now-empty normal_balanced folder
os.rmdir(normal_balanced_dir)
print("Classes merged successfully.")


OSError: [Errno 39] Directory not empty: 'dataset/normal_balanced'

In [53]:
import os
import shutil

# Paths
dataset_dir = "dataset"
normal_dir = os.path.join(dataset_dir, "normal")
normal_balanced_dir = os.path.join(dataset_dir, "normal_balanced")

# Move all images from normal_balanced to normal (skip duplicates)
for file in os.listdir(normal_balanced_dir):
    src_path = os.path.join(normal_balanced_dir, file)
    dst_path = os.path.join(normal_dir, file)
    if not os.path.exists(dst_path):
        shutil.move(src_path, dst_path)

# Remove the normal_balanced folder entirely
shutil.rmtree(normal_balanced_dir)
print("Classes merged successfully.")


Classes merged successfully.


In [54]:
import tensorflow as tf
import os

dataset_dir = "dataset"  # your main dataset folder

# Training dataset (80%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,   # 20% for validation
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Validation dataset (20%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Check class names
print("Classes:", train_ds.class_names)


Found 339 files belonging to 2 classes.
Using 272 files for training.
Found 339 files belonging to 2 classes.
Using 67 files for validation.
Classes: ['normal', 'tumor']


In [55]:
from tensorflow.keras import layers, models

# Define a simple CNN model
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),  # normalize images
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.6272 - loss: 1.4956 - val_accuracy: 0.7612 - val_loss: 0.5932
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - accuracy: 0.7373 - loss: 0.5970 - val_accuracy: 0.7612 - val_loss: 0.6006
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.6988 - loss: 0.6229 - val_accuracy: 0.7612 - val_loss: 0.6170
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 87s 5s/step - accuracy: 0.7096 - loss: 0.6729 - val_accuracy: 0.7612 - val_loss: 0.5542
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.7115 - loss: 0.6019 - val_accuracy: 0.7612 - val_loss: 0.5685
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - accuracy: 0.7027 - loss: 0.6049 - val_accuracy: 0.7612 - val_loss: 0.5617
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 76s 5s/step - accuracy: 0.7430 - loss: 0.5718 - val_accuracy: 0.7612 - val_loss: 0.6261
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.7223 - loss: 0.6132 - val_accuracy: 0.7612 - val_loss: 0.5546
Epoch 9/10
9/9 ━━━━

In [56]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_acc*100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7829 - loss: 0.5825
Validation Accuracy: 76.12%
